### Explore Then Implement

In [2]:
import sys
import os

BASE_DIR = os.getcwd()  # Gets current working directory

sys.path.append(os.path.abspath(os.path.join(BASE_DIR, "..")))

from frontend.transformation import (
    transform_google_locations, transform_google_trend, 
    transform_twitter_hashflags, transform_twitter_locations, transform_twitter_trend
)

In [3]:
import numpy as np

In [5]:
# Assuming transform_twitter_trend() gives you a DataFrame
df = transform_twitter_trend()

# Drop duplicates based on the 'trend' column
df = df.drop_duplicates(subset=['trend'])

# Remove rows where 'meta_description' is missing or empty
df = df[df['meta_description'].notna() & (df['meta_description'] != '')]

# Function to extract the numeric part of the 'meta_description' and handle 'K' and 'M'
def extract_numeric(meta_desc):
    # Remove commas in case there are any in large numbers (e.g., 9,969)
    meta_desc = meta_desc.replace(',', '')
    
    # Try to extract the number from the meta description, ignoring non-numeric characters
    number_str = ''.join(filter(str.isdigit, meta_desc))
    
    # If a number is found, check if there's a suffix (K or M)
    if number_str:
        number = int(number_str)
        
        # Handle 'K' (thousand) and 'M' (million)
        if 'K' in meta_desc.upper():  # 'K' for thousands
            number *= 1000
        elif 'M' in meta_desc.upper():  # 'M' for millions
            number *= 1000000
        return number
    else:
        return np.nan  # Return NaN if no numeric value is found

# Apply the function to create a new column 'meta_description_numeric' for sorting
df['meta_description_numeric'] = df['meta_description'].apply(extract_numeric)

# Sort by the numeric part of the 'meta_description' in descending order
df_sorted = df.sort_values('meta_description_numeric', ascending=False)

# Select only the columns you want to display
df_sorted = df_sorted[['trend', 'meta_description', 'domain_context']]

# Check if the 'meta_description_numeric' column exists before dropping it
if 'meta_description_numeric' in df_sorted.columns:
    df_sorted = df_sorted.drop(columns=['meta_description_numeric'])

# Display the final sorted DataFrame
print(df_sorted)


/Users/maria/Documents/NowTrending/frontend/transformation.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


                   trend       meta_description        domain_context
13142              Trump            2.76M posts              Politics
15112      BREAKING NEWS            1.81M posts                  News
5306                DOGE            1.48M posts  Business and finance
2350               USAID            1.41M posts   Trending in Ukraine
2358                Musk            1.36M posts   Trending in Ukraine
...                  ...                    ...                   ...
13571  Temptation Island            1,007 posts     Trending in Italy
48759           Berlinie            1,002 posts    Trending in Poland
45367      #GenerativeAI            1,001 posts    Trending in Norway
6749              Filiks            1,001 posts    Trending in Poland
2601    #DiversiónBetfun  Promoted by betfun.ok                      

[1372 rows x 3 columns]


In [12]:
def twitter_top_ten():
    # Assuming transform_twitter_trend() gives you a DataFrame
    df = transform_twitter_trend()

    # Drop duplicates based on the 'trend' column
    df = df.drop_duplicates(subset=['trend'])

    # Remove rows where 'meta_description' is missing or empty
    df = df[df['meta_description'].notna() & (df['meta_description'] != '')]

    # Function to extract the numeric part of the 'meta_description' and handle 'K' and 'M'
    def extract_numeric(meta_desc):
        # Remove commas in case there are any in large numbers (e.g., 9,969)
        meta_desc = meta_desc.replace(',', '')
        
        # Try to extract the number from the meta description, ignoring non-numeric characters
        number_str = ''.join(filter(str.isdigit, meta_desc))
        
        # If a number is found, check if there's a suffix (K or M)
        if number_str:
            number = int(number_str)
            
            # Handle 'K' (thousand) and 'M' (million)
            if 'K' in meta_desc.upper():  # 'K' for thousands
                number *= 1000
            elif 'M' in meta_desc.upper():  # 'M' for millions
                number *= 1000000
            return number
        else:
            return np.nan  # Return NaN if no numeric value is found

    # Apply the function to create a new column 'meta_description_numeric' for sorting
    df['meta_description_numeric'] = df['meta_description'].apply(extract_numeric)

    # Sort by the numeric part of the 'meta_description' in descending order
    df_sorted = df.sort_values('meta_description_numeric', ascending=False)

    # Select only the columns you want to display
    df_sorted = df_sorted[['trend', 'meta_description', 'domain_context']]

    # Check if the 'meta_description_numeric' column exists before dropping it
    if 'meta_description_numeric' in df_sorted.columns:
        df_sorted = df_sorted.drop(columns=['meta_description_numeric'])

    # Output the top 10 rows without index
    return df_sorted.head(10)


In [10]:
import pandas as pd

In [15]:
def twitter_trends_location():
    # Transform the location and trend DataFrames
    df_loc = transform_twitter_locations()
    df_trend = transform_twitter_trend()

    # Merge the DataFrames on 'location_id' using an inner join
    merged_df = pd.merge(df_loc, df_trend, on='location_id', how='inner')

    # Select and order the desired columns
    df_sorted = merged_df[['country', 'trend', 'domain_context', 'url']]

    return df_sorted

/Users/maria/Documents/NowTrending/frontend/transformation.py:121: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_unique = pd.read_sql(query, conn)
/Users/maria/Documents/NowTrending/frontend/transformation.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,country,trend,domain_context,url
0,afghanistan,Fantastic Four,Entertainment,twitter://search/?query=%22Fantastic+Four%22&s...
1,afghanistan,Netanyahu,Politics,twitter://search/?query=Netanyahu&src=trend_cl...
2,afghanistan,Jessica Alba,Entertainment,twitter://search/?query=%22Jessica+Alba%22&src...
3,afghanistan,OSHA,Politics,twitter://search/?query=OSHA&src=trend_click&p...
4,afghanistan,Waffle House,Chain restaurants,twitter://search/?query=%22Waffle+House%22&src...
...,...,...,...,...
51315,zimbabwe,Senator Mitch McConnell,Politics,twitter://search/?query=%22Senator+Mitch+McCon...
51316,zimbabwe,Ben Stiller,Entertainment,twitter://search/?query=%22Ben+Stiller%22&src=...
51317,zimbabwe,Gazans,Politics,twitter://search/?query=Gazans&src=trend_click...
51318,zimbabwe,Mila,Politics,twitter://search/?query=Mila&src=trend_click&p...


In [5]:
df = transform_twitter_trend()
df

/Users/maria/Documents/NowTrending/frontend/transformation.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,trend,position,meta_description,domain_context,url,last_updated,location_id
0,1,Fantastic Four,0,,Entertainment,twitter://search/?query=%22Fantastic+Four%22&s...,2025-02-04 22:48:44.048739,-7293673535050703919
1,2,Netanyahu,1,,Politics,twitter://search/?query=Netanyahu&src=trend_cl...,2025-02-04 22:48:44.048739,-7293673535050703919
2,3,Jessica Alba,2,"2,054 posts",Entertainment,twitter://search/?query=%22Jessica+Alba%22&src...,2025-02-04 22:48:44.048739,-7293673535050703919
3,4,OSHA,3,37.9K posts,Politics,twitter://search/?query=OSHA&src=trend_click&p...,2025-02-04 22:48:44.048739,-7293673535050703919
4,5,Waffle House,4,"7,327 posts",Chain restaurants,twitter://search/?query=%22Waffle+House%22&src...,2025-02-04 22:48:44.048739,-7293673535050703919
...,...,...,...,...,...,...,...,...
53444,53445,Martina,7,16.7K posts,Sports,twitter://search/?query=Martina&src=trend_clic...,2025-02-05 23:31:04.686027,3379932461505701466
53445,53446,Arlington,9,"4,342 posts",Music,twitter://search/?query=Arlington&src=trend_cl...,2025-02-05 23:31:04.686027,3379932461505701466
53446,53447,Sundowns,12,"8,877 posts",Sports,twitter://search/?query=Sundowns&src=trend_cli...,2025-02-05 23:31:04.686027,3379932461505701466
53447,53448,Joao Felix,14,30.3K posts,Soccer,twitter://search/?query=%22Joao+Felix%22&src=t...,2025-02-05 23:31:04.686027,3379932461505701466


In [ ]:
# newest trends

# Select only the columns you want to display


def top5_per_context():
    # First, ensure your DataFrame is sorted by last_updated in descending order
    df = transform_twitter_trend()
    df_sorted = df[['trend', 'domain_context', 'last_updated']]
    df_sorted = df_sorted.sort_values('last_updated', ascending=False)

    # Then, for each domain_context group, take the first 5 rows
    top5_per_context = df_sorted.groupby('domain_context').head(5).reset_index(drop=True)

    return top5_per_context

top5_per_context()

/Users/maria/Documents/NowTrending/frontend/transformation.py:96: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,trend,domain_context,last_updated
0,Clinton Foundation,Trending in United States,2025-02-06 00:01:11.374707
1,Delta,Travel,2025-02-06 00:01:11.374707
2,Bob Costas,Sports,2025-02-06 00:01:11.374707
3,Clinton Foundation,Trending in United States,2025-02-06 00:01:10.667905
4,Delta,Travel,2025-02-06 00:01:10.667905
...,...,...,...
514,Arrest,Trending in Washington,2025-02-04 23:01:02.143000
515,Sanji,Television,2025-02-04 23:00:13.811278
516,Marcus Smith,NFL,2025-02-04 22:51:26.668849
517,Sanji,Television,2025-02-04 22:51:25.312332
